## **Data Reading**

In [0]:
df = spark.read.format("parquet")\
    .load("abfss://bronze@sadec25etlproject.dfs.core.windows.net/orders")
df.display()

In [0]:
df.printSchema()

In [0]:
df = df.withColumnRenamed("_rescued_data", "rescued_data")

In [0]:
df = df.drop("rescued_data")
df.display()

In [0]:
df.display()

In [0]:
from pyspark.sql.functions import to_timestamp, col
df = df.withColumn("order_date",to_timestamp(col('order_date')))
df.display()

In [0]:
from pyspark.sql.functions import year
df = df.withColumn("year", year(col("order_date")))
df.display()

In [0]:
df.display()

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import dense_rank, desc
df1 = df.withColumn("flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()


In [0]:
from pyspark.sql.functions import rank
df1 = df1.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

In [0]:
from pyspark.sql.functions import row_number
df1 = df1.withColumn("row_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
df1.display()

## Classes - oop


In [0]:
class windows:
    
    def dense_rank(self, df):
        df_dense_rank = df.withColumn("dense_rank_flag", dense_rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_dense_rank
    
    def rank(self, df):
        df_rank = df.withColumn("rank_flag", rank().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_rank
    
    def row_number(self, df):
        df_row_number = df.withColumn("row_number_flag", row_number().over(Window.partitionBy("year").orderBy(desc("total_amount"))))
        return df_row_number

In [0]:
df_new = df
df_new.display()

In [0]:
obj = windows()

In [0]:
df_result = obj.dense_rank(df_new)
df_result.display()

## **Data Writing**

In [0]:
# in silver layer/ enriched layer data is writen in delta format
df.write.format("delta").mode("overwrite").save("abfss://silver@sadec25etlproject.dfs.core.windows.net/orders")
# instead of append using overwrite, because silverlayer is used as a transient layer don't need to store the data

In [0]:
%sql
CREATE TABLE IF NOT EXISTS dec25etlproject.silver.orders_silver
USING DELTA
LOCATION 'abfss://silver@sadec25etlproject.dfs.core.windows.net/orders'

In [0]:
%sql
SELECT * FROM dec25etlproject.silver.orders_silver;